# Fatigue Detection Using Facial Landmarks and ANN

## Input : coordinates (x,y) of face landmarks 
## Output : Fatigue level


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd


import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Import data analyse class for cacluls parameter from landamrks

In [2]:
from data_analyse import AnalyseData
from format_data import DataFormator

### Define Path of csv data and video infos

In [3]:
video_name = "DESFAM_Semaine 2-Vendredi_PVT_H64"
video_infos_path = "data/stage_data_out/videos_infos.csv"
video_path = "data/stage_data_out/DESFAM_Semaine 2-Vendredi_PVT_H64_hog.csv"

### Append Measure

In [4]:
analyse_data = AnalyseData(video_path)
analyse_data.measure_ear()
analyse_data.df_measure

,frame,ear_l,ear_r,ear
0,0,0.387097,0.415926,0.401511
1,1,0.403226,0.385297,0.394261
2,2,0.401164,0.399806,0.400485
3,3,0.387566,0.399806,0.393686
4,4,0.355385,0.399806,0.377596
...,...,...,...,...
27225,30421,0.401164,0.402635,0.401899
27226,30422,0.403845,0.390164,0.397005
27227,30423,0.419137,0.434648,0.426892
27228,30424,0.403016,0.405671,0.404344


### Append Label

In [5]:
df_ear = DataFormator.make_label_df(num_min = 5, video_name = video_name, df_measure= analyse_data.df_measure)


In [8]:
list(df_ear)
df_temporal, df_label = DataFormator.make_df_temporal_label([10,20,30,40,50],df_ear)
df_temporal

NameError: name 'windows' is not defined

In [7]:
list_frame_consecutive = [10,20,30,40,50]

df_temporal  = pd.DataFrame(columns=list_frame_consecutive, dtype='object')
df_label = pd.DataFrame(columns=list_frame_consecutive)
for num_frame_consecutive in list_frame_consecutive:
    for index in df_ear["frame"]:
        if index+num_frame_consecutive < df_ear["frame"].max():
            ear = df_ear[df_ear["frame"].between(index, index+num_frame_consecutive-1)]["ear"]
            if len(ear)==num_frame_consecutive:
                df_temporal.loc[index,num_frame_consecutive]=list(ear)
                label = 0 if df_ear[df_ear["frame"].between(index, index+num_frame_consecutive-1)]["Target"].sum() == 0 else 1
                df_label = df_label.append(pd.DataFrame([label], columns=[num_frame_consecutive]))
df_temporal

KeyboardInterrupt: 

In [184]:
df_label.to_csv("data/stage_data_out/dataset/label_ear.csv")
df_temporal.to_csv("data/stage_data_out/dataset/features_ear.csv")


10    6860.0
20    6639.0
30    6439.0
40    6249.0
50    6078.0
dtype: float64

In [247]:
list_frame_consecutive = [10,20,30,40,50]
df_tab=[]
for num_frame_consecutive in list_frame_consecutive:    
    df_features = pd.DataFrame(df_temporal[df_temporal[num_frame_consecutive].notna()][num_frame_consecutive]).rename(columns = {num_frame_consecutive : "ear_"+str(num_frame_consecutive)+ "_temporal"})
    df_features = df_features.set_index(np.arange(len(df_features)))
    df_target = pd.DataFrame(df_label[df_label[num_frame_consecutive].notna()][num_frame_consecutive]).rename(columns = {num_frame_consecutive : "target"})
    df_target = df_target.set_index(np.arange(len(df_target)))
    df_tab.append(df_features.join(df_target))


In [230]:
for df in df_tab : 
    df.to_csv("data/stage_data_out/dataset/dataset_"+df.columns[0]+".csv")

NameError: name 'df_tab' is not defined

In [233]:
dataset = tf.data.Dataset.from_tensor_slices((df_tab[0]["ear_10_temporal"].values, df_tab[0]["target"].values))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [61]:
mean_landmarks_normale = pd.DataFrame(df_landmarks[df_landmarks["frame"] <= num_frame_by_num_min].mean())
mean_landmarks_fatigue = pd.DataFrame(df_landmarks[df_landmarks["frame"] > num_frame_by_num_min].mean())
mean_landmarks_fatigue = mean_landmarks_fatigue.drop("frame")
mean_landmarks_normale = mean_landmarks_normale.drop("frame")
mean_landmarks_fatigue = mean_landmarks_fatigue.rename(columns={0 : "Fatigue Mean"})
mean_landmarks_normale = mean_landmarks_normale.rename(columns={0 : "Normale Mean"})


In [63]:
mean_landamrks = pd.concat([mean_landmarks_fatigue, mean_landmarks_normale], axis=1)
mean_landamrks["Variance"] = mean_landamrks.var(axis=1)
mean_landamrks["Mean Difference"] = mean_landamrks["Fatigue Mean"] - mean_landamrks["Normale Mean"]
mean_landamrks

,Fatigue Mean,Normale Mean,Variance,Mean Difference
landmarks_1_x,948.928432,940.268943,37.493377,8.659489
landmarks_1_y,458.070312,458.915694,0.357335,-0.845382
landmarks_2_x,949.041713,941.249086,30.362523,7.792628
landmarks_2_y,478.479771,479.452186,0.472795,-0.972415
landmarks_3_x,950.495117,943.606689,23.725223,6.888428
...,...,...,...,...
landmarks_66_y,536.205078,532.847936,5.635202,3.357142
landmarks_67_x,1024.118025,1020.762237,5.630657,3.355788
landmarks_67_y,536.616490,533.585438,4.593638,3.031052
landmarks_68_x,1017.119141,1013.797074,5.518064,3.322067


In [ ]:
calculer distance relative -> personne change position au cours de la video